In [3]:
import os
import datetime
import pandas as pd
from pandas import json_normalize
import numpy as np
import re
import json
import requests
import time
from bs4 import BeautifulSoup
from collections import defaultdict, Counter
import random
import pprint

C:\Users\ebbi_\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


1) Get links to different categories of recipes from https://www.allrecipes.com/recipes/
  - JS path document.querySelector("#mntl-taxonomysc-header__bottom_1-0")
2) For each category, get links to recipes
3) Scrape each recipe for ingredients and cooking instructions

In [4]:
import os
import requests
import pprint

#Vivian API 5a0c4c04c9694535b01b6beabca5230a
#Ebad API 9a3d7f53cd9a46c2b756f580a317df46
#Jiaqi API bcf31b839c1c40e99af333f1510201af

vivian_url = "https://api.spoonacular.com/recipes/complexSearch?addRecipeInformation=true&apiKey=5a0c4c04c9694535b01b6beabca5230a"
#ebad_url = "https://api.spoonacular.com/food/search?apiKey=9a3d7f53cd9a46c2b756f580a317df46"
#jiaqi_url = "https://api.spoonacular.com/food/menuItems/search?apiKey=bcf31b839c1c40e99af333f1510201af"


spoonacular_response_vivian = requests.get(vivian_url)
#spoonacular_response_jiaqi = requests.get(jiaqi_url)
#spoonacular_response_ebad = requests.get(ebad_url)

if spoonacular_response_vivian.status_code == 200:
    pprint.pprint(spoonacular_response_vivian.json())
else:
    print(f"Error: {spoonacular_response_vivian.status_code}")


{'number': 10,
 'offset': 0,
 'results': [{'aggregateLikes': 309,
              'analyzedInstructions': [{'name': '',
                                        'steps': [{'equipment': [],
                                                   'ingredients': [{'id': 10716050,
                                                                    'image': 'cooked-cannellini-beans.png',
                                                                    'localizedName': 'cannellini '
                                                                                     'beans',
                                                                    'name': 'cannellini '
                                                                            'beans'},
                                                                   {'id': 14412,
                                                                    'image': 'water.png',
                                                                    'localizedName

store it into a file

In [5]:
# load results into a Pandas dataframe
data=spoonacular_response_vivian.json()
df=pd.DataFrame(data)

# parse data
recipes = pd.json_normalize(df['results'])
recipes


,vegetarian,vegan,glutenFree,dairyFree,veryHealthy,cheap,veryPopular,sustainable,lowFodmap,weightWatcherSmartPoints,...,imageType,summary,cuisines,dishTypes,diets,occasions,analyzedInstructions,spoonacularScore,spoonacularSourceUrl,license
0,True,True,True,True,True,False,False,False,False,12,...,jpg,Cannellini Bean and Asparagus Salad with Mushr...,[],"[side dish, lunch, salad, main course, main di...","[gluten free, dairy free, lacto ovo vegetarian...",[],"[{'name': '', 'steps': [{'number': 1, 'step': ...",99.535843,https://spoonacular.com/cannellini-bean-and-as...,NaN
1,True,False,False,False,True,False,True,False,False,15,...,jpg,If you want to add more <b>lacto ovo vegetaria...,[],"[morning meal, brunch, beverage, breakfast, dr...",[lacto ovo vegetarian],[],"[{'name': '', 'steps': [{'number': 1, 'step': ...",99.463066,https://spoonacular.com/berry-banana-breakfast...,NaN
2,False,False,True,True,True,False,True,False,False,11,...,jpg,Red Lentil Soup with Chicken and Turnips might...,[],"[soup, lunch, main course, main dish, dinner]","[gluten free, dairy free]","[fall, winter]","[{'name': '', 'steps': [{'number': 1, 'step': ...",99.404068,https://spoonacular.com/red-lentil-soup-with-c...,NaN
3,True,True,True,True,True,False,False,False,False,2,...,jpg,Asparagus and Pea Soup: Real Convenience Food ...,[],"[soup, antipasti, starter, snack, appetizer, a...","[gluten free, dairy free, paleolithic, lacto o...","[fall, winter]","[{'name': '', 'steps': [{'number': 1, 'step': ...",99.392303,https://spoonacular.com/asparagus-and-pea-soup...,CC BY-SA 3.0
4,True,True,True,True,True,False,False,False,False,5,...,jpg,Garlicky Kale requires approximately <b>45 min...,[],[side dish],"[gluten free, dairy free, paleolithic, lacto o...",[],"[{'name': '', 'steps': [{'number': 1, 'step': ...",98.892021,https://spoonacular.com/garlicky-kale-644387,CC BY 3.0
5,False,False,True,True,True,False,False,False,False,10,...,jpg,If you want to add more <b>gluten free and dai...,[],"[soup, lunch, main course, main dish, dinner]","[gluten free, dairy free]","[fall, winter]","[{'name': '', 'steps': [{'number': 1, 'step': ...",98.790100,https://spoonacular.com/slow-cooker-beef-stew-...,NaN
6,True,True,True,True,True,False,False,False,False,9,...,jpg,Red Kidney Bean Jambalayan is a main course th...,"[Creole, Cajun]","[lunch, main course, main dish, dinner]","[gluten free, dairy free, lacto ovo vegetarian...",[],"[{'name': '', 'steps': [{'number': 1, 'step': ...",98.757645,https://spoonacular.com/red-kidney-bean-jambal...,NaN
7,False,False,True,False,True,False,False,False,False,12,...,jpg,Chicken Fajita Stuffed Bell Pepper takes appro...,[Mexican],"[lunch, main course, main dish, dinner]",[gluten free],[],"[{'name': '', 'steps': [{'number': 1, 'step': ...",98.739464,https://spoonacular.com/chicken-fajita-stuffed...,NaN
8,True,True,True,True,False,False,True,False,False,7,...,jpg,"The recipe Cauliflower, Brown Rice, and Vegeta...","[Chinese, Asian]","[side dish, antipasti, starter, snack, appetiz...","[gluten free, dairy free, lacto ovo vegetarian...",[],"[{'name': '', 'steps': [{'number': 1, 'step': ...",98.649734,https://spoonacular.com/cauliflower-brown-rice...,CC BY-SA 3.0
9,True,True,True,True,True,False,False,False,False,23,...,jpg,You can never have too many middl eastern reci...,[Middle Eastern],"[seasoning, marinade]","[gluten free, dairy free, lacto ovo vegetarian...",[],"[{'name': '', 'steps': [{'number': 1, 'step': ...",98.601486,https://spoonacular.com/hummus-and-zaatar-766453,CC BY 2.5 CA


In [6]:
# ingredients/instructions are nested under 'analyzedInstructions'
recipes.columns

Index(['vegetarian', 'vegan', 'glutenFree', 'dairyFree', 'veryHealthy',
       'cheap', 'veryPopular', 'sustainable', 'lowFodmap',
       'weightWatcherSmartPoints', 'gaps', 'preparationMinutes',
       'cookingMinutes', 'aggregateLikes', 'healthScore', 'creditsText',
       'sourceName', 'pricePerServing', 'id', 'title', 'readyInMinutes',
       'servings', 'sourceUrl', 'image', 'imageType', 'summary', 'cuisines',
       'dishTypes', 'diets', 'occasions', 'analyzedInstructions',
       'spoonacularScore', 'spoonacularSourceUrl', 'license'],
      dtype='object')

In [7]:
recipes

,vegetarian,vegan,glutenFree,dairyFree,veryHealthy,cheap,veryPopular,sustainable,lowFodmap,weightWatcherSmartPoints,...,imageType,summary,cuisines,dishTypes,diets,occasions,analyzedInstructions,spoonacularScore,spoonacularSourceUrl,license
0,True,True,True,True,True,False,False,False,False,12,...,jpg,Cannellini Bean and Asparagus Salad with Mushr...,[],"[side dish, lunch, salad, main course, main di...","[gluten free, dairy free, lacto ovo vegetarian...",[],"[{'name': '', 'steps': [{'number': 1, 'step': ...",99.535843,https://spoonacular.com/cannellini-bean-and-as...,NaN
1,True,False,False,False,True,False,True,False,False,15,...,jpg,If you want to add more <b>lacto ovo vegetaria...,[],"[morning meal, brunch, beverage, breakfast, dr...",[lacto ovo vegetarian],[],"[{'name': '', 'steps': [{'number': 1, 'step': ...",99.463066,https://spoonacular.com/berry-banana-breakfast...,NaN
2,False,False,True,True,True,False,True,False,False,11,...,jpg,Red Lentil Soup with Chicken and Turnips might...,[],"[soup, lunch, main course, main dish, dinner]","[gluten free, dairy free]","[fall, winter]","[{'name': '', 'steps': [{'number': 1, 'step': ...",99.404068,https://spoonacular.com/red-lentil-soup-with-c...,NaN
3,True,True,True,True,True,False,False,False,False,2,...,jpg,Asparagus and Pea Soup: Real Convenience Food ...,[],"[soup, antipasti, starter, snack, appetizer, a...","[gluten free, dairy free, paleolithic, lacto o...","[fall, winter]","[{'name': '', 'steps': [{'number': 1, 'step': ...",99.392303,https://spoonacular.com/asparagus-and-pea-soup...,CC BY-SA 3.0
4,True,True,True,True,True,False,False,False,False,5,...,jpg,Garlicky Kale requires approximately <b>45 min...,[],[side dish],"[gluten free, dairy free, paleolithic, lacto o...",[],"[{'name': '', 'steps': [{'number': 1, 'step': ...",98.892021,https://spoonacular.com/garlicky-kale-644387,CC BY 3.0
5,False,False,True,True,True,False,False,False,False,10,...,jpg,If you want to add more <b>gluten free and dai...,[],"[soup, lunch, main course, main dish, dinner]","[gluten free, dairy free]","[fall, winter]","[{'name': '', 'steps': [{'number': 1, 'step': ...",98.790100,https://spoonacular.com/slow-cooker-beef-stew-...,NaN
6,True,True,True,True,True,False,False,False,False,9,...,jpg,Red Kidney Bean Jambalayan is a main course th...,"[Creole, Cajun]","[lunch, main course, main dish, dinner]","[gluten free, dairy free, lacto ovo vegetarian...",[],"[{'name': '', 'steps': [{'number': 1, 'step': ...",98.757645,https://spoonacular.com/red-kidney-bean-jambal...,NaN
7,False,False,True,False,True,False,False,False,False,12,...,jpg,Chicken Fajita Stuffed Bell Pepper takes appro...,[Mexican],"[lunch, main course, main dish, dinner]",[gluten free],[],"[{'name': '', 'steps': [{'number': 1, 'step': ...",98.739464,https://spoonacular.com/chicken-fajita-stuffed...,NaN
8,True,True,True,True,False,False,True,False,False,7,...,jpg,"The recipe Cauliflower, Brown Rice, and Vegeta...","[Chinese, Asian]","[side dish, antipasti, starter, snack, appetiz...","[gluten free, dairy free, lacto ovo vegetarian...",[],"[{'name': '', 'steps': [{'number': 1, 'step': ...",98.649734,https://spoonacular.com/cauliflower-brown-rice...,CC BY-SA 3.0
9,True,True,True,True,True,False,False,False,False,23,...,jpg,You can never have too many middl eastern reci...,[Middle Eastern],"[seasoning, marinade]","[gluten free, dairy free, lacto ovo vegetarian...",[],"[{'name': '', 'steps': [{'number': 1, 'step': ...",98.601486,https://spoonacular.com/hummus-and-zaatar-766453,CC BY 2.5 CA


In [8]:
# extract ingredients and instructions, using aggregateLikes as dish identifier
ingredients=pd.json_normalize(df['results'], record_path=['analyzedInstructions','steps'], meta='aggregateLikes')
ingredients


,number,step,ingredients,equipment,length.number,length.unit,aggregateLikes
0,1,Rinse the cannellini beans and soak for 8 hour...,"[{'id': 10716050, 'name': 'cannellini beans', ...",[],480.0,minutes,309
1,2,"Drain and rinse, then transfer to a medium sau...","[{'id': 14412, 'name': 'water', 'localizedName...","[{'id': 404669, 'name': 'sauce pan', 'localize...",NaN,NaN,309
2,3,Add the curry leaves or bay leaf and bring to ...,"[{'id': 93604, 'name': 'curry leaves', 'locali...",[],60.0,minutes,309
3,4,Drain and transfer to a large salad bowl.Meanw...,"[{'id': 11011, 'name': 'asparagus', 'localized...","[{'id': 404783, 'name': 'bowl', 'localizedName...",6.0,minutes,309
4,5,Transfer to the salad bowl.Now cook the mushro...,"[{'id': 11260, 'name': 'mushrooms', 'localized...","[{'id': 404783, 'name': 'bowl', 'localizedName...",NaN,NaN,309
5,6,Heat the oil in a large skillet over high heat...,"[{'id': 11260, 'name': 'mushrooms', 'localized...","[{'id': 404645, 'name': 'frying pan', 'localiz...",5.0,minutes,309
6,7,Transfer to the bowl with the asparagus.To mak...,"[{'id': 9152, 'name': 'lemon juice', 'localize...","[{'id': 404771, 'name': 'food processor', 'loc...",NaN,NaN,309
7,8,"Pour the dressing over the salad, season with ...","[{'id': 1102047, 'name': 'salt and pepper', 'l...",[],NaN,NaN,309
8,9,Serve at room temperature or chilled.,[],[],NaN,NaN,309
9,1,Take some yogurt in your favorite flavor and a...,"[{'id': 1116, 'name': 'yogurt', 'localizedName...","[{'id': 404726, 'name': 'blender', 'localizedN...",NaN,NaN,689


In [9]:
# Combine steps for the same dish
combined_steps = ingredients.groupby(['aggregateLikes'], as_index=False)['step'].agg(lambda x: ' '.join(x))
combined_steps

,aggregateLikes,step
0,19,Heat the olive oil in a large pot over medium ...
1,35,Rinse the chickpeas and soak for 8 hours or ov...
2,53,Rinse the kidney beans and brown rice separate...
3,57,"To get started, heat your slow cooker to low. ..."
4,159,"To get started heat oven to 35 Mix salt, peppe..."
5,207,Chop the garlic and onions. Saute the onions i...
6,309,Rinse the cannellini beans and soak for 8 hour...
7,689,Take some yogurt in your favorite flavor and a...
8,1866,"To a large dutch oven or soup pot, heat the ol..."
9,3689,Remove the cauliflower's tough stem and reserv...


In [10]:
# need to extract ingredients and combine rows for each dish
merged_df = pd.merge(combined_steps['aggregateLikes'], ingredients[['aggregateLikes', 'ingredients', 'step']], on='aggregateLikes')
merged_df = merged_df.groupby('aggregateLikes', as_index=False).agg({
    'ingredients': lambda x: [i for sublist in x.dropna() for i in sublist],
    'step': ' '.join
})

# merge back w/ original recipes df
full_df = pd.merge(recipes,merged_df, on='aggregateLikes')

In [11]:
# remove irrelevant columns
final_df = full_df[['ingredients','step','title']]

# rearrange to have title be the first column
column_to_assign_first = 'title'
final_df = final_df[['title'] + [col for col in final_df.columns if col != 'title']]


In [12]:
final_df

,title,ingredients,step
0,Cannellini Bean and Asparagus Salad with Mushr...,"[{'id': 10716050, 'name': 'cannellini beans', ...",Rinse the cannellini beans and soak for 8 hour...
1,Berry Banana Breakfast Smoothie,"[{'id': 1116, 'name': 'yogurt', 'localizedName...",Take some yogurt in your favorite flavor and a...
2,Red Lentil Soup with Chicken and Turnips,"[{'id': 4053, 'name': 'olive oil', 'localizedN...","To a large dutch oven or soup pot, heat the ol..."
3,Asparagus and Pea Soup: Real Convenience Food,"[{'id': 11215, 'name': 'garlic', 'localizedNam...",Chop the garlic and onions. Saute the onions i...
4,Garlicky Kale,"[{'id': 4053, 'name': 'olive oil', 'localizedN...",Heat the olive oil in a large pot over medium ...
5,Slow Cooker Beef Stew,"[{'id': 6147, 'name': 'cream of mushroom soup'...","To get started, heat your slow cooker to low. ..."
6,Red Kidney Bean Jambalaya,"[{'id': 16033, 'name': 'kidney beans', 'locali...",Rinse the kidney beans and brown rice separate...
7,Chicken Fajita Stuffed Bell Pepper,"[{'id': 2009, 'name': 'chili powder', 'localiz...","To get started heat oven to 35 Mix salt, peppe..."
8,"Cauliflower, Brown Rice, and Vegetable Fried Rice","[{'id': 10011135, 'name': 'cauliflower florets...",Remove the cauliflower's tough stem and reserv...
9,Hummus and Za'atar,"[{'id': 16057, 'name': 'chickpeas', 'localized...",Rinse the chickpeas and soak for 8 hours or ov...


In [26]:
final_df.iloc[0]['ingredients']

[{'id': 10716050,
  'name': 'cannellini beans',
  'localizedName': 'cannellini beans',
  'image': 'cooked-cannellini-beans.png'},
 {'id': 14412,
  'name': 'water',
  'localizedName': 'water',
  'image': 'water.png'},
 {'id': 14412,
  'name': 'water',
  'localizedName': 'water',
  'image': 'water.png'},
 {'id': 93604,
  'name': 'curry leaves',
  'localizedName': 'curry leaves',
  'image': 'curry-leaves.jpg'},
 {'id': 2004,
  'name': 'bay leaves',
  'localizedName': 'bay leaves',
  'image': 'bay-leaves.jpg'},
 {'id': 0,
  'name': 'beans',
  'localizedName': 'beans',
  'image': 'kidney-beans.jpg'},
 {'id': 11011,
  'name': 'asparagus',
  'localizedName': 'asparagus',
  'image': 'asparagus.png'},
 {'id': 11260,
  'name': 'mushrooms',
  'localizedName': 'mushrooms',
  'image': 'mushrooms.png'},
 {'id': 11260,
  'name': 'mushrooms',
  'localizedName': 'mushrooms',
  'image': 'mushrooms.png'},
 {'id': 4582,
  'name': 'cooking oil',
  'localizedName': 'cooking oil',
  'image': 'vegetable-oil.j

In [29]:


df_test_ebad = pd.DataFrame(final_df[['ingredients']])
df_test_ebad


,ingredients
0,"[{'id': 10716050, 'name': 'cannellini beans', ..."
1,"[{'id': 1116, 'name': 'yogurt', 'localizedName..."
2,"[{'id': 4053, 'name': 'olive oil', 'localizedN..."
3,"[{'id': 11215, 'name': 'garlic', 'localizedNam..."
4,"[{'id': 4053, 'name': 'olive oil', 'localizedN..."
5,"[{'id': 6147, 'name': 'cream of mushroom soup'..."
6,"[{'id': 16033, 'name': 'kidney beans', 'locali..."
7,"[{'id': 2009, 'name': 'chili powder', 'localiz..."
8,"[{'id': 10011135, 'name': 'cauliflower florets..."
9,"[{'id': 16057, 'name': 'chickpeas', 'localized..."


In [30]:
df_list_ebad = []

for i in range(len(df_test_ebad)):
    df_list_ebad.append(pd.DataFrame([df_test_ebad.iloc[i]]))

df_list_name = []

for i in df_list_ebad:
    df_list_name.append(i['name'].tolist()[0])
 
df_list_name



KeyError: 'name'